## Import các thư viện

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

## Tạo dữ liệu mẫu

In [2]:
data = {
    'customer': ['Nguyen Van A', 'Nguyen Van A', 'Le Thi B', 'Le Thi B', 'Tran Van C', 'Tran Van C', 'Nguyen Van D', 'Nguyen Van D'],
    'product': ['Cà phê sữa', 'Đen Đá', 'Trà sữa', 'Cà phê sữa', 'Đen Đá', 'Trà sữa', 'Cà phê sữa', 'Trà sữa'],
    'amount': [1, 3, 2, 1, 2, 1, 1, 2],
    'date': ['2024-09-16', '2024-09-16', '2024-09-15', '2024-09-15', '2024-09-14', '2024-09-14', '2024-09-13', '2024-09-13']
}

In [3]:
df = pd.DataFrame(data)

In [4]:
df.head()

,customer,product,amount,date
0,Nguyen Van A,Cà phê sữa,1,2024-09-16
1,Nguyen Van A,Đen Đá,3,2024-09-16
2,Le Thi B,Trà sữa,2,2024-09-15
3,Le Thi B,Cà phê sữa,1,2024-09-15
4,Tran Van C,Đen Đá,2,2024-09-14


## Tạo ma trận khách hàng - sản phẩm

In [5]:
customer_product_matrix = df.pivot_table(index='customer', columns='product', values='amount', fill_value=0)


## Tính toán độ tương đồng giữa các khách hàng

In [6]:
customer_similarity = cosine_similarity(customer_product_matrix)

## Chuyển thành DataFrame để dễ quan sát

In [7]:
customer_similarity_df = pd.DataFrame(customer_similarity, index=customer_product_matrix.index, columns=customer_product_matrix.index)


In [8]:
customer_similarity_df.head()

customer,Le Thi B,Nguyen Van A,Nguyen Van D,Tran Van C
customer,,,,
Le Thi B,1.000000,0.141421,1.000000,0.400000
Nguyen Van A,0.141421,1.000000,0.141421,0.848528
Nguyen Van D,1.000000,0.141421,1.000000,0.400000
Tran Van C,0.400000,0.848528,0.400000,1.000000


## Hàm gợi ý sản phẩm cho khách hàng dựa trên độ tương đồng

In [13]:
def recommend_products(customer, customer_product_matrix, similarity_df, num_recommendations=2):
    # Lấy thông tin sản phẩm mà khách hàng đã mua
    customer_purchases = customer_product_matrix.loc[customer]
    
    # Tìm các khách hàng tương tự
    similar_customers = similarity_df[customer].sort_values(ascending=False).index
    
    # Tập hợp các sản phẩm mà khách hàng tương tự đã mua nhưng khách hàng hiện tại chưa mua
    recommended_products = set()
    
    for similar_customer in similar_customers:
        if similar_customer != customer:
            similar_customer_purchases = customer_product_matrix.loc[similar_customer]
            # Sản phẩm mà khách hàng tương tự đã mua nhưng khách hàng hiện tại chưa mua
            products_to_recommend = similar_customer_purchases[(similar_customer_purchases > 0) & (customer_purchases == 0)]
            
            recommended_products.update(products_to_recommend.index)
    
    # Chỉ trả về tên sản phẩm
    return list(recommended_products)[:num_recommendations]

## Gợi ý sản phẩm cho khách hàng

In [14]:
# Gợi ý sản phẩm cho "Nguyen Van A"
recommendations_for_nguyen_van_a = recommend_products("Nguyen Van A", customer_product_matrix, customer_similarity_df)
print("Gợi ý sản phẩm cho Nguyen Van A:", recommendations_for_nguyen_van_a)

# Gợi ý sản phẩm cho "Le Thi B"
recommendations_for_le_thi_b = recommend_products("Le Thi B", customer_product_matrix, customer_similarity_df)
print("Gợi ý sản phẩm cho Le Thi B:", recommendations_for_le_thi_b)

Gợi ý sản phẩm cho Nguyen Van A: ['Trà sữa']
Gợi ý sản phẩm cho Le Thi B: ['Đen Đá']
